# OECD HAN Database

In [1]:
# load packages 
library(pacman)
p_load(tidyverse, data.table, dtplyr, reshape2, archive, kableExtra)
# set option
options(dplyr.summarise.inform = FALSE)
gray_scale <- c('#F3F4F8','#D2D4DA', '#B3B5BD', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

In [2]:
han_names <- fread('work/notebooks/patent/data/202208_HAN_NAMES.txt')

In [7]:
dim(han_names)

[1] 4191007       3

In [7]:
head(han_names)

HAN_ID,Clean_name,Person_ctry_code
<int>,<chr>,<chr>
1,& HAMBOURG NIENDORF,DE
2,& KK,JP
3,“ASTRONIT” CLOSE CORP,RU
4,“DEUTSCHE SEE” GMBH,DE
5,“EFIRNOIE” OPEN JOINT STOCK CO,RU
6,“EUROSTANDART” LTD LIABILITY CO,RU


In [3]:
de_firms <- fread('work/notebooks/patent/data/orbis_de_matched_l.csv')

In [55]:
han_names %>%
    .[1:5,] %>%
    kable('pipe', align='ccc')



| HAN_ID |           Clean_name           | Person_ctry_code |
|:------:|:------------------------------:|:----------------:|
|   1    |      & HAMBOURG NIENDORF       |        DE        |
|   2    |              & KK              |        JP        |
|   3    |     “ASTRONIT” CLOSE CORP      |        RU        |
|   4    |      “DEUTSCHE SEE” GMBH       |        DE        |
|   5    | “EFIRNOIE” OPEN JOINT STOCK CO |        RU        |

In [53]:
de_firms %>%
    .[1:5, 2:3]

name_native,name_internat
<chr>,<chr>
Airbus Defence and Space GmbH,Airbus Defence and Space GmbH
EurA AG,EurA AG
TuTech Innovation GmbH,TuTech Innovation GmbH
FFT Produktionssysteme GmbH & Co. KG.,FFT Produktionssysteme GmbH & Co. KG.
Diehl Aviation Laupheim GmbH,Diehl Aviation Laupheim GmbH


In [4]:
# query
airbus <- toupper('Airbus Defence')
han_names %>%
    .[Person_ctry_code == 'DE'] %>%
    .[Clean_name %like% airbus] %>%
    .[,HAN_ID] -> airbus_han_ids
airbus_han_ids

[1]   60513   62422 3637004 4401227 4527012

In [5]:
han_person <- fread('work/notebooks/patent/data/202208_HAN_PERSON.txt')

In [6]:
han_patents <- fread('work/notebooks/patent/data/202208_HAN_PATENTS.txt')

In [19]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[sample(.N, 5)]

HAN_ID,HARM_ID,Appln_id,Publn_auth,Patent_number
<int>,<int>,<int>,<chr>,<chr>
60513,60513,156990,EP,EP1920908
60513,60513,483683605,US,US10703486
60513,60513,481847610,EP,EP3435476
60513,60513,415684469,US,US9308691
60513,60513,419500061,EP,EP2960636


In [26]:
fooids = c('US2017165795', 'EP3181711', 'US10703486', 'EP3219614', 'US9308691','EP2689872')
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[Patent_number %in% fooids] %>%
    kable('pipe', align='ccccl')



| HAN_ID | HARM_ID | Appln_id  | Publn_auth |Patent_number |
|:------:|:-------:|:---------:|:----------:|:-------------|
| 60513  |  60513  | 408666014 |     EP     |EP2689872     |
| 60513  |  60513  | 415684469 |     US     |US9308691     |
| 60513  |  60513  | 450308171 |     EP     |EP3219614     |
| 60513  |  60513  | 480021212 |     US     |US2017165795  |
| 60513  |  60513  | 483683605 |     US     |US10703486    |

In [34]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[Publn_auth == 'EP'] %>%
    head %>%
    kable('pipe', align='ccccc')



| HAN_ID | HARM_ID | Appln_id | Publn_auth | Patent_number |
|:------:|:-------:|:--------:|:----------:|:-------------:|
| 60513  |  60513  |   213    |     EP     |   EP2030891   |
| 60513  |  60513  |  65448   |     EP     |   EP2025928   |
| 60513  |  60513  |  156990  |     EP     |   EP1920908   |
| 60513  |  60513  |  161551  |     EP     |   EP1972896   |
| 60513  |  60513  |  173385  |     EP     |   EP2134522   |
| 60513  |  60513  |  173386  |     EP     |   EP2136979   |